# Pythonでpingの実装してみる
> "Writing simple ping example"

- toc: true
- categories: [Python, Network]

## はじめに
ネットワークの疎通確認を行う際など，pingコマンドにお世話になっている方は多いと思います．私も勿論その一人です．
ですが，実際pingは何をしているのか？という点は前々理解していませんでした．そこで，pythonにてpingのサブセット（エコー要求とエコー応答のみ）を実装してみました．

## やったこと

* Pythonでpingコマンドのサブセットを実装
* エコー要求に対してエコー応答が返信されることを確認

## ICMPについて
ICMPとはInternet Control Message Protocolの略称です．名前の通り，インターネットの通信に関する情報の送信に用いられます．
pingコマンドは，このプロトコルのエコー要求とエコー応答という二つのメッセージをやりとりすることで，端末間の疎通確認を行います．

エコー要求とエコー応答のパケットフォーマットは以下の通りです．

![svg image](data:image/svg+xml,%3Csvg%20viewBox%3D%220%200%20896%20408%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20xmlns%3Ainkspace%3D%22http%3A%2F%2Fwww.inkscape.org%2Fnamespaces%2Finkscape%22%20xmlns%3Axlink%3D%22http%3A%2F%2Fwww.w3.org%2F1999%2Fxlink%22%3E%0A%20%20%3Cdefs%20id%3D%22defs_block%22%3E%0A%20%20%20%20%3Cfilter%20height%3D%221.504%22%20id%3D%22filter_blur%22%20inkspace%3Acollect%3D%22always%22%20width%3D%221.1575%22%20x%3D%22-0.07875%22%20y%3D%22-0.252%22%3E%0A%20%20%20%20%20%20%3CfeGaussianBlur%20id%3D%22feGaussianBlur3780%22%20inkspace%3Acollect%3D%22always%22%20stdDeviation%3D%224.2%22%20%2F%3E%0A%20%20%20%20%3C%2Ffilter%3E%0A%20%20%3C%2Fdefs%3E%0A%20%20%3Ctitle%3Eblockdiag%3C%2Ftitle%3E%0A%20%20%3Cdesc%3E%7B%0A%20%20colwidth%20%3D%2032%0A%20%20node_height%20%3D%2072%0A%0A%20%200-7%3A%20Type%0A%20%208-15%3A%20Code%0A%20%2016-31%3A%20Checksum%0A%20%2032-47%3A%20Identification%0A%20%2048-63%3A%20Sequence%20Number%0A%20%2064-95%3A%20Data%5Bcolheight%20%3D%202%5D%0A%7D%0A%3C%2Fdesc%3E%0A%20%20%3Cpath%20d%3D%22M%2064%2048%20L%2064%2080%22%20fill%3D%22none%22%20stroke%3D%22rgb%280%2C0%2C0%29%22%20%2F%3E%0A%20%20%3Ctext%20fill%3D%22rgb%280%2C0%2C0%29%22%20font-family%3D%22sans-serif%22%20font-size%3D%2211%22%20font-style%3D%22normal%22%20font-weight%3D%22normal%22%20text-anchor%3D%22middle%22%20textLength%3D%227%22%20x%3D%2264.5%22%20y%3D%2242%22%3E0%3C%2Ftext%3E%0A%20%20%3Cpath%20d%3D%22M%2088%2064%20L%2088%2080%22%20fill%3D%22none%22%20stroke%3D%22rgb%280%2C0%2C0%29%22%20%2F%3E%0A%20%20%3Cpath%20d%3D%22M%20112%2064%20L%20112%2080%22%20fill%3D%22none%22%20stroke%3D%22rgb%280%2C0%2C0%29%22%20%2F%3E%0A%20%20%3Cpath%20d%3D%22M%20136%2064%20L%20136%2080%22%20fill%3D%22none%22%20stroke%3D%22rgb%280%2C0%2C0%29%22%20%2F%3E%0A%20%20%3Cpath%20d%3D%22M%20160%2064%20L%20160%2080%22%20fill%3D%22none%22%20stroke%3D%22rgb%280%2C0%2C0%29%22%20%2F%3E%0A%20%20%3Cpath%20d%3D%22M%20184%2064%20L%20184%2080%22%20fill%3D%22none%22%20stroke%3D%22rgb%280%2C0%2C0%29%22%20%2F%3E%0A%20%20%3Cpath%20d%3D%22M%20208%2064%20L%20208%2080%22%20fill%3D%22none%22%20stroke%3D%22rgb%280%2C0%2C0%29%22%20%2F%3E%0A%20%20%3Cpath%20d%3D%22M%20232%2064%20L%20232%2080%22%20fill%3D%22none%22%20stroke%3D%22rgb%280%2C0%2C0%29%22%20%2F%3E%0A%20%20%3Cpath%20d%3D%22M%20256%2048%20L%20256%2080%22%20fill%3D%22none%22%20stroke%3D%22rgb%280%2C0%2C0%29%22%20%2F%3E%0A%20%20%3Cpath%20d%3D%22M%20280%2064%20L%20280%2080%22%20fill%3D%22none%22%20stroke%3D%22rgb%280%2C0%2C0%29%22%20%2F%3E%0A%20%20%3Cpath%20d%3D%22M%20304%2064%20L%20304%2080%22%20fill%3D%22none%22%20stroke%3D%22rgb%280%2C0%2C0%29%22%20%2F%3E%0A%20%20%3Cpath%20d%3D%22M%20328%2064%20L%20328%2080%22%20fill%3D%22none%22%20stroke%3D%22rgb%280%2C0%2C0%29%22%20%2F%3E%0A%20%20%3Cpath%20d%3D%22M%20352%2064%20L%20352%2080%22%20fill%3D%22none%22%20stroke%3D%22rgb%280%2C0%2C0%29%22%20%2F%3E%0A%20%20%3Cpath%20d%3D%22M%20376%2064%20L%20376%2080%22%20fill%3D%22none%22%20stroke%3D%22rgb%280%2C0%2C0%29%22%20%2F%3E%0A%20%20%3Cpath%20d%3D%22M%20400%2064%20L%20400%2080%22%20fill%3D%22none%22%20stroke%3D%22rgb%280%2C0%2C0%29%22%20%2F%3E%0A%20%20%3Cpath%20d%3D%22M%20424%2064%20L%20424%2080%22%20fill%3D%22none%22%20stroke%3D%22rgb%280%2C0%2C0%29%22%20%2F%3E%0A%20%20%3Cpath%20d%3D%22M%20448%2048%20L%20448%2080%22%20fill%3D%22none%22%20stroke%3D%22rgb%280%2C0%2C0%29%22%20%2F%3E%0A%20%20%3Ctext%20fill%3D%22rgb%280%2C0%2C0%29%22%20font-family%3D%22sans-serif%22%20font-size%3D%2211%22%20font-style%3D%22normal%22%20font-weight%3D%22normal%22%20text-anchor%3D%22middle%22%20textLength%3D%2215%22%20x%3D%22448.5%22%20y%3D%2242%22%3E16%3C%2Ftext%3E%0A%20%20%3Cpath%20d%3D%22M%20472%2064%20L%20472%2080%22%20fill%3D%22none%22%20stroke%3D%22rgb%280%2C0%2C0%29%22%20%2F%3E%0A%20%20%3Cpath%20d%3D%22M%20496%2064%20L%20496%2080%22%20fill%3D%22none%22%20stroke%3D%22rgb%280%2C0%2C0%29%22%20%2F%3E%0A%20%20%3Cpath%20d%3D%22M%20520%2064%20L%20520%2080%22%20fill%3D%22none%22%20stroke%3D%22rgb%280%2C0%2C0%29%22%20%2F%3E%0A%20%20%3Cpath%20d%3D%22M%20544%2064%20L%20544%2080%22%20fill%3D%22none%22%20stroke%3D%22rgb%280%2C0%2C0%29%22%20%2F%3E%0A%20%20%3Cpath%20d%3D%22M%20568%2064%20L%20568%2080%22%20fill%3D%22none%22%20stroke%3D%22rgb%280%2C0%2C0%29%22%20%2F%3E%0A%20%20%3Cpath%20d%3D%22M%20592%2064%20L%20592%2080%22%20fill%3D%22none%22%20stroke%3D%22rgb%280%2C0%2C0%29%22%20%2F%3E%0A%20%20%3Cpath%20d%3D%22M%20616%2064%20L%20616%2080%22%20fill%3D%22none%22%20stroke%3D%22rgb%280%2C0%2C0%29%22%20%2F%3E%0A%20%20%3Cpath%20d%3D%22M%20640%2048%20L%20640%2080%22%20fill%3D%22none%22%20stroke%3D%22rgb%280%2C0%2C0%29%22%20%2F%3E%0A%20%20%3Cpath%20d%3D%22M%20664%2064%20L%20664%2080%22%20fill%3D%22none%22%20stroke%3D%22rgb%280%2C0%2C0%29%22%20%2F%3E%0A%20%20%3Cpath%20d%3D%22M%20688%2064%20L%20688%2080%22%20fill%3D%22none%22%20stroke%3D%22rgb%280%2C0%2C0%29%22%20%2F%3E%0A%20%20%3Cpath%20d%3D%22M%20712%2064%20L%20712%2080%22%20fill%3D%22none%22%20stroke%3D%22rgb%280%2C0%2C0%29%22%20%2F%3E%0A%20%20%3Cpath%20d%3D%22M%20736%2064%20L%20736%2080%22%20fill%3D%22none%22%20stroke%3D%22rgb%280%2C0%2C0%29%22%20%2F%3E%0A%20%20%3Cpath%20d%3D%22M%20760%2064%20L%20760%2080%22%20fill%3D%22none%22%20stroke%3D%22rgb%280%2C0%2C0%29%22%20%2F%3E%0A%20%20%3Cpath%20d%3D%22M%20784%2064%20L%20784%2080%22%20fill%3D%22none%22%20stroke%3D%22rgb%280%2C0%2C0%29%22%20%2F%3E%0A%20%20%3Cpath%20d%3D%22M%20808%2064%20L%20808%2080%22%20fill%3D%22none%22%20stroke%3D%22rgb%280%2C0%2C0%29%22%20%2F%3E%0A%20%20%3Cpath%20d%3D%22M%20832%2048%20L%20832%2080%22%20fill%3D%22none%22%20stroke%3D%22rgb%280%2C0%2C0%29%22%20%2F%3E%0A%20%20%3Ctext%20fill%3D%22rgb%280%2C0%2C0%29%22%20font-family%3D%22sans-serif%22%20font-size%3D%2211%22%20font-style%3D%22normal%22%20font-weight%3D%22normal%22%20text-anchor%3D%22middle%22%20textLength%3D%2215%22%20x%3D%22832.5%22%20y%3D%2242%22%3E32%3C%2Ftext%3E%0A%20%20%3Crect%20fill%3D%22rgb%28255%2C255%2C255%29%22%20height%3D%2272%22%20stroke%3D%22rgb%28255%2C255%2C255%29%22%20width%3D%22192%22%20x%3D%2264%22%20y%3D%2280%22%20%2F%3E%0A%20%20%3Cpath%20d%3D%22M%2064%2080%20L%20256%2080%22%20fill%3D%22none%22%20stroke%3D%22rgb%280%2C0%2C0%29%22%20%2F%3E%0A%20%20%3Cpath%20d%3D%22M%20256%2080%20L%20256%20152%22%20fill%3D%22none%22%20stroke%3D%22rgb%280%2C0%2C0%29%22%20%2F%3E%0A%20%20%3Cpath%20d%3D%22M%20256%20152%20L%2064%20152%22%20fill%3D%22none%22%20stroke%3D%22rgb%280%2C0%2C0%29%22%20%2F%3E%0A%20%20%3Cpath%20d%3D%22M%2064%20152%20L%2064%2080%22%20fill%3D%22none%22%20stroke%3D%22rgb%280%2C0%2C0%29%22%20%2F%3E%0A%20%20%3Ctext%20fill%3D%22rgb%280%2C0%2C0%29%22%20font-family%3D%22sans-serif%22%20font-size%3D%2211%22%20font-style%3D%22normal%22%20font-weight%3D%22normal%22%20text-anchor%3D%22middle%22%20textLength%3D%2226%22%20x%3D%22160.0%22%20y%3D%22122%22%3EType%3C%2Ftext%3E%0A%20%20%3Crect%20fill%3D%22rgb%28255%2C255%2C255%29%22%20height%3D%2272%22%20stroke%3D%22rgb%28255%2C255%2C255%29%22%20width%3D%22192%22%20x%3D%22256%22%20y%3D%2280%22%20%2F%3E%0A%20%20%3Cpath%20d%3D%22M%20256%2080%20L%20448%2080%22%20fill%3D%22none%22%20stroke%3D%22rgb%280%2C0%2C0%29%22%20%2F%3E%0A%20%20%3Cpath%20d%3D%22M%20448%2080%20L%20448%20152%22%20fill%3D%22none%22%20stroke%3D%22rgb%280%2C0%2C0%29%22%20%2F%3E%0A%20%20%3Cpath%20d%3D%22M%20448%20152%20L%20256%20152%22%20fill%3D%22none%22%20stroke%3D%22rgb%280%2C0%2C0%29%22%20%2F%3E%0A%20%20%3Cpath%20d%3D%22M%20256%20152%20L%20256%2080%22%20fill%3D%22none%22%20stroke%3D%22rgb%280%2C0%2C0%29%22%20%2F%3E%0A%20%20%3Ctext%20fill%3D%22rgb%280%2C0%2C0%29%22%20font-family%3D%22sans-serif%22%20font-size%3D%2211%22%20font-style%3D%22normal%22%20font-weight%3D%22normal%22%20text-anchor%3D%22middle%22%20textLength%3D%2226%22%20x%3D%22352.0%22%20y%3D%22122%22%3ECode%3C%2Ftext%3E%0A%20%20%3Crect%20fill%3D%22rgb%28255%2C255%2C255%29%22%20height%3D%2272%22%20stroke%3D%22rgb%28255%2C255%2C255%29%22%20width%3D%22384%22%20x%3D%22448%22%20y%3D%2280%22%20%2F%3E%0A%20%20%3Cpath%20d%3D%22M%20448%2080%20L%20832%2080%22%20fill%3D%22none%22%20stroke%3D%22rgb%280%2C0%2C0%29%22%20%2F%3E%0A%20%20%3Cpath%20d%3D%22M%20832%2080%20L%20832%20152%22%20fill%3D%22none%22%20stroke%3D%22rgb%280%2C0%2C0%29%22%20%2F%3E%0A%20%20%3Cpath%20d%3D%22M%20832%20152%20L%20448%20152%22%20fill%3D%22none%22%20stroke%3D%22rgb%280%2C0%2C0%29%22%20%2F%3E%0A%20%20%3Cpath%20d%3D%22M%20448%20152%20L%20448%2080%22%20fill%3D%22none%22%20stroke%3D%22rgb%280%2C0%2C0%29%22%20%2F%3E%0A%20%20%3Ctext%20fill%3D%22rgb%280%2C0%2C0%29%22%20font-family%3D%22sans-serif%22%20font-size%3D%2211%22%20font-style%3D%22normal%22%20font-weight%3D%22normal%22%20text-anchor%3D%22middle%22%20textLength%3D%2252%22%20x%3D%22640.0%22%20y%3D%22122%22%3EChecksum%3C%2Ftext%3E%0A%20%20%3Crect%20fill%3D%22rgb%28255%2C255%2C255%29%22%20height%3D%2272%22%20stroke%3D%22rgb%28255%2C255%2C255%29%22%20width%3D%22384%22%20x%3D%2264%22%20y%3D%22152%22%20%2F%3E%0A%20%20%3Cpath%20d%3D%22M%2064%20152%20L%20448%20152%22%20fill%3D%22none%22%20stroke%3D%22rgb%280%2C0%2C0%29%22%20%2F%3E%0A%20%20%3Cpath%20d%3D%22M%20448%20152%20L%20448%20224%22%20fill%3D%22none%22%20stroke%3D%22rgb%280%2C0%2C0%29%22%20%2F%3E%0A%20%20%3Cpath%20d%3D%22M%20448%20224%20L%2064%20224%22%20fill%3D%22none%22%20stroke%3D%22rgb%280%2C0%2C0%29%22%20%2F%3E%0A%20%20%3Cpath%20d%3D%22M%2064%20224%20L%2064%20152%22%20fill%3D%22none%22%20stroke%3D%22rgb%280%2C0%2C0%29%22%20%2F%3E%0A%20%20%3Ctext%20fill%3D%22rgb%280%2C0%2C0%29%22%20font-family%3D%22sans-serif%22%20font-size%3D%2211%22%20font-style%3D%22normal%22%20font-weight%3D%22normal%22%20text-anchor%3D%22middle%22%20textLength%3D%2261%22%20x%3D%22256.5%22%20y%3D%22194%22%3EIdentification%3C%2Ftext%3E%0A%20%20%3Crect%20fill%3D%22rgb%28255%2C255%2C255%29%22%20height%3D%2272%22%20stroke%3D%22rgb%28255%2C255%2C255%29%22%20width%3D%22384%22%20x%3D%22448%22%20y%3D%22152%22%20%2F%3E%0A%20%20%3Cpath%20d%3D%22M%20448%20152%20L%20832%20152%22%20fill%3D%22none%22%20stroke%3D%22rgb%280%2C0%2C0%29%22%20%2F%3E%0A%20%20%3Cpath%20d%3D%22M%20832%20152%20L%20832%20224%22%20fill%3D%22none%22%20stroke%3D%22rgb%280%2C0%2C0%29%22%20%2F%3E%0A%20%20%3Cpath%20d%3D%22M%20832%20224%20L%20448%20224%22%20fill%3D%22none%22%20stroke%3D%22rgb%280%2C0%2C0%29%22%20%2F%3E%0A%20%20%3Cpath%20d%3D%22M%20448%20224%20L%20448%20152%22%20fill%3D%22none%22%20stroke%3D%22rgb%280%2C0%2C0%29%22%20%2F%3E%0A%20%20%3Ctext%20fill%3D%22rgb%280%2C0%2C0%29%22%20font-family%3D%22sans-serif%22%20font-size%3D%2211%22%20font-style%3D%22normal%22%20font-weight%3D%22normal%22%20text-anchor%3D%22middle%22%20textLength%3D%2293%22%20x%3D%22640.5%22%20y%3D%22194%22%3ESequence%20Number%3C%2Ftext%3E%0A%20%20%3Crect%20fill%3D%22rgb%28255%2C255%2C255%29%22%20height%3D%22144%22%20stroke%3D%22rgb%28255%2C255%2C255%29%22%20width%3D%22768%22%20x%3D%2264%22%20y%3D%22224%22%20%2F%3E%0A%20%20%3Cpath%20d%3D%22M%2064%20224%20L%20832%20224%22%20fill%3D%22none%22%20stroke%3D%22rgb%280%2C0%2C0%29%22%20%2F%3E%0A%20%20%3Cpath%20d%3D%22M%20832%20224%20L%20832%20368%22%20fill%3D%22none%22%20stroke%3D%22rgb%280%2C0%2C0%29%22%20%2F%3E%0A%20%20%3Cpath%20d%3D%22M%20832%20368%20L%2064%20368%22%20fill%3D%22none%22%20stroke%3D%22rgb%280%2C0%2C0%29%22%20%2F%3E%0A%20%20%3Cpath%20d%3D%22M%2064%20368%20L%2064%20224%22%20fill%3D%22none%22%20stroke%3D%22rgb%280%2C0%2C0%29%22%20%2F%3E%0A%20%20%3Ctext%20fill%3D%22rgb%280%2C0%2C0%29%22%20font-family%3D%22sans-serif%22%20font-size%3D%2211%22%20font-style%3D%22normal%22%20font-weight%3D%22normal%22%20text-anchor%3D%22middle%22%20textLength%3D%2224%22%20x%3D%22448.0%22%20y%3D%22302%22%3EData%3C%2Ftext%3E%0A%3C%2Fsvg%3E%0A)

なお，これらのメッセージはIPデータグラムのペイロードとして送受信されます．

## Pythonによる実装
### エコー要求とエコー応答に対応するデータ構造
上述のパケットフォーマットにおける`Type`は，エコー要求では`8`が，エコー応答では`0`が設定されます．
従って，以下のように`ICMPType`として定義することができます．

In [24]:
from enum import Enum

class ICMPType(Enum):
    ECHOREPLY = 0
    ECHO = 8

    def __int__(self):
        return self.value

また，パケットフォーマットからチェックサムフィールドを確認することができます．
このチェックサムは以下の手順で計算します．

1. チェックサムフィールドを`0`で埋める
1. パケットを16ビット単位で区切る（パケット長が奇数バイトである場合は`0x00`を末尾に追加）
1. 16ビット単位で区切ったデータを1の補数で加算していき合計値を求める
1. 加算結果の1の補数を最終結果とする．この時，0の表現としては`0xffff`を用いる．

以上の手順でチェックサムを計算する関数`calc_checksum`を以下のように定義します．

In [25]:
def calc_checksum(data: bytes) -> int:
    if len(data) % 2 == 1:
        data += b"\x00"
    u16_counts = len(data) // 2
    checksum = sum(struct.unpack(f"!{u16_counts}H", data))
    while 0xFFFF < checksum:
        checksum = (checksum & 0xFFFF) + (checksum >> 16)
    if checksum != 0xFFFF:
        checksum = ~checksum
    return checksum & 0xFFFF

以下に示す`ICMPEcho`がエコー要求とエコー応答を表現するデータ構造となります．`checksum`がコンストラクタ引数として指定されなかった場合は，`calc_checksum`にて計算します．

`ICMPEcho`はネットワークを通して送受信されます．従って，シリアライズを行う`to_bytes`と，デシリアライズを行う`from_bytes`を実装しています．

In [26]:
from dataclasses import dataclass
from typing import Optional
import struct

@dataclass(frozen=True)
class ICMPEcho:
    type: ICMPType
    code: int
    id: int
    seq: int
    data: bytes
    checksum: Optional[int] = None

    def __post_init__(self):
        if self.checksum is None:
            object.__setattr__(self, "checksum", 0)
            object.__setattr__(self, "checksum", calc_checksum(self.to_bytes()))

    def to_bytes(self) -> bytes:
        return struct.pack(
            f"!BBHHH{len(self.data)}s",
            int(self.type),
            self.code,
            self.checksum,
            self.id,
            self.seq,
            self.data,
        )

    @classmethod
    def from_bytes(cls, packed: bytes) -> "ICMPEcho":
        _type, code, checksum, id, seq = struct.unpack("!BBHHH", packed[:8])
        type = ICMPType(_type)
        data = packed[8:]
        return ICMPEcho(type, code, id, seq, data, checksum=checksum)

### Rawソケットの作成
TCPやUDPを用いた時と同様，ICMPでもソケットを用いてネットワークプログラミングを行います．
しかしながら，ICMPパケットはIPデータグラムのペイロードとするため，IPデータグラムを操作する必要があります．

IPデータグラムの操作はRawソケットを用いることで実現できます．そこで，ソケットを作成する関数`raw_socket`を以下のように定義しました．
`socket.socket`の第２引数には`socket.SOCK_RAW`を第３引数には`socket.IPPROTO_ICMP`を指定します．これは，RawソケットをICMPから利用することを表していいます．

In [27]:
import socket
from contextlib import contextmanager

@contextmanager
def raw_socket():
    sock = socket.socket(socket.AF_INET, socket.SOCK_RAW, socket.IPPROTO_ICMP)
    try:
        yield sock
    finally:
        sock.close()

### IPデータグラムのパース
Rawソケットによって受信したデータはIPデータグラムです．従って，これを適切にパースし，エコー応答をペイロードとして取り出す必要があります．
今回，以下のようにIPヘッダを表現する`IPHeader`と，IPデータグラムをパースする`parse_ip_datagram`関数を実装しました．
`parse_ip_datagram`関数では，受信したデータの先頭２０バイトから`IPHeader`インスタンスを作成し，残りのデータをペイロードとします．

In [28]:
from typing import Tuple

@dataclass(frozen=True)
class IPHeader:
    v: int
    hl: int
    tos: int
    len: int
    id: int
    off: int
    ttl: int
    p: int
    sum: int
    src: str
    dst: str

    @staticmethod
    def from_bytes(packed: bytes) -> "IPHeader":
        v_hl, tos, len, id, off, ttl, p, sum, src, dst = struct.unpack(
            "!BBHHHBBHII", packed
        )
        v = v_hl >> 4
        hl = v_hl & 0x0F

        return IPHeader(
            v,
            hl,
            tos,
            len,
            id,
            off,
            ttl,
            p,
            sum,
            socket.inet_ntoa(src.to_bytes(4, byteorder="big")),
            socket.inet_ntoa(dst.to_bytes(4, byteorder="big")),
        )

def parse_ip_datagram(data: bytes) -> Tuple[IPHeader, bytes]:
    ip_header = IPHeader.from_bytes(data[:20])
    payload = data[20:]
    return (ip_header, payload)

### エコー要求の送信とエコー応答の受信
これまで実装してきた関数とデータ構造を用いて，エコー要求を送信しエコー応答を受信する関数`ping`を実装しました．
以下に示されるように，エコー要求をソケットに書き込み，ソケットから読み取ったエコー応答とIPヘッダを出力します．

In [29]:
def print_response(ip_header: IPHeader, echo_reply: ICMPEcho) -> None:
    print(
        f"ping echo reply from {ip_header.src}: icmp_seq={echo_reply.seq} ttl={ip_header.ttl}"
    )


def ping(host: str, seq: int) -> None:
    with raw_socket() as sock:
        packet = ICMPEcho(ICMPType.ECHO, 0, 0, seq, b"\xff").to_bytes()
        sock.sendto(packet, (host, 0))
        ip_header, payload = parse_ip_datagram(sock.recvfrom(4096)[0])
        echo_reply = ICMPEcho.from_bytes(payload)
        print_response(ip_header, echo_reply)

### 実行結果
実際に`8.8.8.8`に対して`ping`を呼び出した結果を以下に示します．
エコー要求に対して適切なエコー応答が帰ってきていることが確認できます．

In [30]:
import time
for i in range(10):
    ping("8.8.8.8", i)
    time.sleep(1)

ping echo reply from 8.8.8.8: icmp_seq=0 ttl=37
ping echo reply from 8.8.8.8: icmp_seq=1 ttl=37
ping echo reply from 8.8.8.8: icmp_seq=2 ttl=37
ping echo reply from 8.8.8.8: icmp_seq=3 ttl=37
ping echo reply from 8.8.8.8: icmp_seq=4 ttl=37
ping echo reply from 8.8.8.8: icmp_seq=5 ttl=37
ping echo reply from 8.8.8.8: icmp_seq=6 ttl=37
ping echo reply from 8.8.8.8: icmp_seq=7 ttl=37
ping echo reply from 8.8.8.8: icmp_seq=8 ttl=37
ping echo reply from 8.8.8.8: icmp_seq=9 ttl=37


## 参考
* 村山公保，2004年，基礎からわかるTCP/IP ネットワーク実験プログラミング（第2版），オーム社